In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class DNANeuron:
    def __init__(self, input_size):
        self.weights = np.random.randn(input_size)
        self.bias = np.random.randn()

    def activate(self, inputs):
        return 1 if np.dot(self.weights, inputs) + self.bias > 0 else 0

class DNANetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_layer = [DNANeuron(input_size) for _ in range(hidden_size)]
        self.output_neuron = DNANeuron(hidden_size)

    def predict(self, X):
        hidden_outputs = [neuron.activate(X) for neuron in self.hidden_layer]
        return self.output_neuron.activate(hidden_outputs)

def create_population(pop_size, input_size, hidden_size, output_size):
    return [DNANetwork(input_size, hidden_size, output_size) for _ in range(pop_size)]

def fitness(network, X, y):
    predictions = [network.predict(x) for x in X]
    return accuracy_score(y, predictions)

def select_parents(population, fitnesses, num_parents):
    return [population[i] for i in np.argsort(fitnesses)[-num_parents:]]

def crossover(parent1, parent2):
    child = DNANetwork(len(parent1.hidden_layer[0].weights), len(parent1.hidden_layer), 1)
    for i in range(len(child.hidden_layer)):
        if np.random.rand() < 0.5:
            child.hidden_layer[i] = parent1.hidden_layer[i]
        else:
            child.hidden_layer[i] = parent2.hidden_layer[i]
    if np.random.rand() < 0.5:
        child.output_neuron = parent1.output_neuron
    else:
        child.output_neuron = parent2.output_neuron
    return child

def mutate(network, mutation_rate):
    for neuron in network.hidden_layer + [network.output_neuron]:
        if np.random.rand() < mutation_rate:
            neuron.weights += np.random.normal(0, 0.1, size=neuron.weights.shape)
            neuron.bias += np.random.normal(0, 0.1)

def evolve(X_train, y_train, generations=100, pop_size=50, hidden_size=5, mutation_rate=0.1):
    input_size = X_train.shape[1]
    population = create_population(pop_size, input_size, hidden_size, 1)

    for gen in range(generations):
        fitnesses = [fitness(net, X_train, y_train) for net in population]
        best_fitness = max(fitnesses)
        print(f"Generation {gen + 1}, Best Fitness: {best_fitness}")

        parents = select_parents(population, fitnesses, num_parents=pop_size // 2)

        next_generation = parents.copy()
        while len(next_generation) < pop_size:
            parent1, parent2 = np.random.choice(parents, 2, replace=False)
            child = crossover(parent1, parent2)
            mutate(child, mutation_rate)
            next_generation.append(child)

        population = next_generation

    return population[np.argmax([fitness(net, X_train, y_train) for net in population])]

In [2]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [5]:
# Load the Breast Cancer Wisconsin dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Preprocess the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train the DNA-inspired neural network
print("Training DNA-inspired Neural Network...")
best_network = evolve(X_train, y_train, generations=10, pop_size=100, hidden_size=10, mutation_rate=0.1)

# Evaluate the DNA-inspired neural network
dna_train_predictions = [best_network.predict(x) for x in X_train]
dna_test_predictions = [best_network.predict(x) for x in X_test]

print("\nDNA-inspired Neural Network Results:")
print(f"Train Accuracy: {accuracy_score(y_train, dna_train_predictions):.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, dna_test_predictions):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, dna_test_predictions))

# Train and evaluate Random Forest
print("\nTraining Random Forest...")
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

rf_train_predictions = rf_classifier.predict(X_train)
rf_test_predictions = rf_classifier.predict(X_test)

print("\nRandom Forest Results:")
print(f"Train Accuracy: {accuracy_score(y_train, rf_train_predictions):.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, rf_test_predictions):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, rf_test_predictions))

# Compare the results
print("\nComparison:")
print(f"DNA-inspired NN Test Accuracy: {accuracy_score(y_test, dna_test_predictions):.4f}")
print(f"Random Forest Test Accuracy: {accuracy_score(y_test, rf_test_predictions):.4f}")

Training DNA-inspired Neural Network...
Generation 1, Best Fitness: 0.9032967032967033
Generation 2, Best Fitness: 0.8989010989010989
Generation 3, Best Fitness: 0.8945054945054945
Generation 4, Best Fitness: 0.8967032967032967
Generation 5, Best Fitness: 0.8989010989010989
Generation 6, Best Fitness: 0.8989010989010989
Generation 7, Best Fitness: 0.8989010989010989
Generation 8, Best Fitness: 0.9054945054945055
Generation 9, Best Fitness: 0.9340659340659341
Generation 10, Best Fitness: 0.9406593406593406

DNA-inspired Neural Network Results:
Train Accuracy: 0.9385
Test Accuracy: 0.9211

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.86      0.89        43
           1       0.92      0.96      0.94        71

    accuracy                           0.92       114
   macro avg       0.92      0.91      0.91       114
weighted avg       0.92      0.92      0.92       114


Training Random Forest...

Random Forest Results:
Trai